# Zgradite regresijski model: priprava in vizualizacija podatkov

## **Linearna regresija za buče - Lekcija 2**
#### Uvod

Zdaj, ko imate na voljo orodja, ki jih potrebujete za začetek gradnje modelov strojnega učenja z Tidymodels in Tidyverse, ste pripravljeni začeti postavljati vprašanja o svojih podatkih. Ko delate s podatki in uporabljate rešitve strojnega učenja, je zelo pomembno, da znate postaviti prava vprašanja, da pravilno izkoristite potencial svojega nabora podatkov.

V tej lekciji boste spoznali:

-   Kako pripraviti podatke za gradnjo modela.

-   Kako uporabiti `ggplot2` za vizualizacijo podatkov.

Vprašanje, na katerega želite dobiti odgovor, bo določilo, katere vrste algoritmov strojnega učenja boste uporabili. Kakovost odgovora, ki ga dobite, pa bo močno odvisna od narave vaših podatkov.

Poglejmo to skozi praktično vajo.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Umetniško delo @allison_horst</figcaption>


<!--![Umetniško delo \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Umetniško delo \@allison_horst-->


## 1. Uvoz podatkov o bučah in priklic Tidyverse

Za obdelavo te lekcije bomo potrebovali naslednje pakete:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [zbirka paketov za R](https://www.tidyverse.org/packages), zasnovana za hitrejše, enostavnejše in bolj zabavno podatkovno znanost!

Namestite jih lahko z ukazom:

`install.packages(c("tidyverse"))`

Spodnji skript preveri, ali imate nameščene pakete, potrebne za dokončanje tega modula, in jih namesti, če kateri manjka.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Zdaj zaženimo nekaj paketov in naložimo [podatke](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), ki so na voljo za to lekcijo!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Hitro `glimpse()` takoj pokaže, da so prisotne prazne vrednosti ter mešanica nizov (`chr`) in numeričnih podatkov (`dbl`). `Date` je tipa znak, poleg tega pa je tu še nenavadni stolpec z imenom `Package`, kjer so podatki mešanica med `sacks`, `bins` in drugimi vrednostmi. Podatki so, pravzaprav, precej zmedeni 😤.

Pravzaprav ni ravno pogosto, da bi dobili podatkovni niz, ki je popolnoma pripravljen za uporabo pri ustvarjanju modela strojnega učenja kar takoj. A brez skrbi, v tej lekciji se boste naučili, kako pripraviti surov podatkovni niz z uporabo standardnih knjižnic v R 🧑‍🔧. Prav tako se boste naučili različnih tehnik za vizualizacijo podatkov.📈📊
<br>

> Osvežitev: Operator cevi (`%>%`) izvaja operacije v logičnem zaporedju tako, da objekt posreduje naprej v funkcijo ali izraz klica. Operator cevi si lahko predstavljate kot "in potem" v vaši kodi.


## 2. Preverjanje manjkajočih podatkov

Ena najpogostejših težav, s katerimi se soočajo podatkovni znanstveniki, so nepopolni ali manjkajoči podatki. R predstavlja manjkajoče ali neznane vrednosti s posebnim označevalcem: `NA` (Not Available).

Kako torej ugotovimo, da podatkovni okvir vsebuje manjkajoče vrednosti?
<br>
-   Eden od preprostih načinov je uporaba osnovne R funkcije `anyNA`, ki vrne logične vrednosti `TRUE` ali `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Odlično, zdi se, da manjkajo nekateri podatki! To je dobro izhodišče.

-   Druga možnost bi bila uporaba funkcije `is.na()`, ki pokaže, kateri posamezni elementi stolpcev manjkajo, z logično vrednostjo `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

V redu, naloga je opravljena, vendar bi bilo z veliko podatkovno tabelo, kot je ta, neučinkovito in praktično nemogoče pregledati vse vrstice in stolpce posamično😴.

-   Bolj intuitiven način bi bil izračunati vsoto manjkajočih vrednosti za vsak stolpec:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Veliko bolje! Manjkajo podatki, vendar morda to ne bo pomembno za nalogo. Poglejmo, kaj bo prinesla nadaljnja analiza.

> Poleg odličnih paketov in funkcij ima R zelo dobro dokumentacijo. Na primer, uporabite `help(colSums)` ali `?colSums`, da izveste več o funkciji.


## 3. Dplyr: Slovnica za manipulacijo podatkov

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ilustracija avtorice @allison_horst</figcaption>


<!--![Ilustracija avtorice \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Ilustracija avtorice \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), paket v Tidyverse, je slovnica za obdelavo podatkov, ki ponuja dosleden nabor glagolov, s katerimi lahko rešite najpogostejše izzive pri obdelavi podatkov. V tem razdelku bomo raziskali nekatere glagole dplyr!


#### dplyr::select()

`select()` je funkcija v paketu `dplyr`, ki vam pomaga izbrati stolpce za ohranitev ali izključitev.

Da bo vaš podatkovni okvir lažji za delo, odstranite več njegovih stolpcev z uporabo `select()` in obdržite samo tiste stolpce, ki jih potrebujete.

Na primer, v tej vaji bo naša analiza vključevala stolpce `Package`, `Low Price`, `High Price` in `Date`. Izberimo te stolpce.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` je funkcija v paketu `dplyr`, ki vam omogoča ustvarjanje ali spreminjanje stolpcev, pri čemer obstoječi stolpci ostanejo nespremenjeni.

Splošna struktura funkcije mutate je:

`data %>%   mutate(new_column_name = what_it_contains)`

Poglejmo, kako deluje `mutate`, z uporabo stolpca `Date` in izvedbo naslednjih operacij:

1.  Pretvorba datumov (trenutno tipa znak) v format meseca (to so ameriški datumi, zato je format `MM/DD/YYYY`).

2.  Izvleček meseca iz datumov v nov stolpec.

V jeziku R paket [lubridate](https://lubridate.tidyverse.org/) olajša delo s podatki tipa Datum-čas. Zato uporabimo `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` in preverimo, kako doseči zgoraj navedene cilje. Stolpec Date lahko odstranimo, saj ga v nadaljnjih operacijah ne bomo več potrebovali.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Juhu! 🤩

Zdaj ustvarimo nov stolpec `Price`, ki predstavlja povprečno ceno buče. Nato izračunajmo povprečje stolpcev `Low Price` in `High Price`, da zapolnimo novi stolpec Price.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Ja!💪

"Počakaj malo!", boš rekel po hitrem pregledu celotnega nabora podatkov z `View(pumpkins)`, "Tukaj je nekaj nenavadnega!"🤔

Če pogledaš stolpec `Package`, so buče prodane v različnih konfiguracijah. Nekatere so prodane v merah `1 1/9 bushel`, nekatere v merah `1/2 bushel`, nekatere na bučo, nekatere na funt, in nekatere v velikih škatlah z različnimi širinami.

Preverimo to:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Neverjetno!👏

Buče je očitno zelo težko dosledno tehtati, zato jih filtrirajmo tako, da izberemo samo buče z nizom *bushel* v stolpcu `Package` in jih shranimo v nov podatkovni okvir `new_pumpkins`.


#### dplyr::filter() in stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): ustvari podmnožico podatkov, ki vsebuje **vrstice**, ki ustrezajo vašim pogojem, v tem primeru buče z nizom *bushel* v stolpcu `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): zazna prisotnost ali odsotnost vzorca v nizu.

Paket [`stringr`](https://github.com/tidyverse/stringr) ponuja enostavne funkcije za pogoste operacije z nizi.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Vidite, da smo zožili izbor na približno 415 vrstic podatkov, ki vsebujejo buče po koših.🤩


#### dplyr::case_when()

**Ampak počakajte! Še nekaj je treba narediti**

Ste opazili, da se količina košev razlikuje po vrsticah? Potrebno je normalizirati cene, da bodo prikazane na koš, ne pa na 1 1/9 ali 1/2 koša. Čas je za nekaj matematike, da to standardiziramo.

Uporabili bomo funkcijo [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), da *spremenimo* stolpec Price glede na določene pogoje. `case_when` omogoča vektorizacijo več `if_else()` stavkov.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Zdaj lahko analiziramo ceno na enoto glede na njihovo meritev v bušlnih. Vsa ta študija o bušlnih buč pa kaže, kako zelo `pomembno` je `razumeti naravo svojih podatkov`!

> ✅ Po navedbah [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) teža bušlja zavisi od vrste pridelka, saj gre za meritev prostornine. "Bušelj paradižnikov, na primer, naj bi tehtal 56 funtov... Listi in zelenjava zavzamejo več prostora z manjšo težo, zato bušelj špinače tehta le 20 funtov." Vse skupaj je precej zapleteno! Ne ukvarjajmo se s pretvorbo iz bušljev v funte, ampak raje določimo ceno po bušlju. Vsa ta študija o bušlnih buč pa kaže, kako zelo pomembno je razumeti naravo svojih podatkov!
>
> ✅ Ste opazili, da so buče, prodane po pol bušlja, zelo drage? Ali lahko ugotovite, zakaj? Namig: majhne buče so precej dražje od velikih, verjetno zato, ker jih je v bušlju veliko več, glede na neizkoriščen prostor, ki ga zavzame ena velika votla buča za pito.


Pomembno pravilo:  
1. NE dodajajte '''markdown ali kakršnih koli drugih oznak okoli prevoda  
2. Poskrbite, da prevod ne bo zvenel preveč dobesedno  
3. Prevedite tudi komentarje  
4. Ta datoteka je napisana v obliki Markdown - ne obravnavajte je kot XML ali HTML  
5. Ne prevajajte:  
   - [!NOTE], [!WARNING], [!TIP], [!IMPORTANT], [!CAUTION]  
   - Imen spremenljivk, funkcij, razredov  
   - Oznak, kot so @@INLINE_CODE_x@@ ali @@CODE_BLOCK_x@@  
   - URL-jev ali poti  
6. Ohranite vso izvirno oblikovanje Markdown nespremenjeno  
7. Vrni SAMO prevedeno vsebino brez dodatnih oznak ali oblikovanja  

Zdaj pa, za čisto avanturo 💁‍♀️, premaknimo stolpec Month na prvo mesto, torej `pred` stolpec `Package`.  

Za spremembo položajev stolpcev se uporablja `dplyr::relocate()`.  


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Odlično delo!👌 Zdaj imate čist in urejen nabor podatkov, na katerem lahko zgradite svoj novi regresijski model!  
<br>


## 4. Vizualizacija podatkov z ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografika avtorja Dasani Madipalli</figcaption>


<!--![Infografika avtorja Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Obstaja *modri* rek, ki pravi:

> "Preprost graf je analitiku podatkov prinesel več informacij kot katerakoli druga naprava." --- John Tukey

Del vloge podatkovnega znanstvenika je prikazati kakovost in naravo podatkov, s katerimi dela. To pogosto dosežejo z ustvarjanjem zanimivih vizualizacij, kot so grafi, diagrami in prikazi, ki pokažejo različne vidike podatkov. Na ta način lahko vizualno prikažejo odnose in vrzeli, ki jih je sicer težko odkriti.

Vizualizacije lahko pomagajo tudi pri določanju najbolj primerne tehnike strojnega učenja za podatke. Na primer, razpršen diagram, ki sledi črti, nakazuje, da so podatki primerni za nalogo linearne regresije.

R ponuja več sistemov za izdelavo grafov, vendar je [`ggplot2`](https://ggplot2.tidyverse.org/index.html) eden najbolj elegantnih in vsestranskih. `ggplot2` omogoča sestavljanje grafov z **združevanjem neodvisnih komponent**.

Začnimo s preprostim razpršenim diagramom za stolpca Price in Month.

V tem primeru bomo začeli z [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), podali podatkovni niz in estetsko preslikavo (z [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), nato pa dodali sloje (kot je [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) za razpršene diagrame.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Je to uporaben graf 🤷? Ali te kaj na njem preseneča?

Ni posebej uporaben, saj zgolj prikazuje tvoje podatke kot razpored točk v določenem mesecu.
<br>


### **Kako naredimo podatke uporabne?**

Da bi prikazali koristne podatke na grafikonih, je običajno potrebno podatke nekako združiti. Na primer, v našem primeru bi iskanje povprečne cene buč za vsak mesec omogočilo boljši vpogled v osnovne vzorce v naših podatkih. To nas pripelje do še ene hitre predstavitve **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Združeno agregiranje v R lahko enostavno izvedemo z

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` spremeni enoto analize iz celotnega nabora podatkov na posamezne skupine, kot so na primer meseci.

-   `dplyr::summarize()` ustvari nov podatkovni okvir z enim stolpcem za vsako spremenljivko skupine in enim stolpcem za vsako statistiko povzetka, ki ste jo določili.

Na primer, lahko uporabimo `dplyr::group_by() %>% summarize()` za združevanje buč v skupine na podlagi stolpca **Month** in nato izračunamo **povprečno ceno** za vsak mesec.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Jedrnato!✨

Kategorijske značilnosti, kot so meseci, so bolje prikazane z uporabo stolpčnega diagrama 📊. Plasti, ki so odgovorne za stolpčne diagrame, so `geom_bar()` in `geom_col()`. Preverite `?geom_bar` za več informacij.

Pa ga ustvarimo!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 To je bolj uporabna vizualizacija podatkov! Zdi se, da kaže, da so najvišje cene buč v septembru in oktobru. Ali to ustreza vašim pričakovanjem? Zakaj ali zakaj ne?

Čestitke za zaključek druge lekcije 👏! Pripravili ste svoje podatke za gradnjo modela in nato odkrili več vpogledov z uporabo vizualizacij!



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
